# Public Transporation Heatmap using Google Directions API

In [ ]:
import gmaps
import googlemaps
from datetime import datetime
import math
import time
import random # for random shuffle
import pickle # for loading and storing of data 
import numpy # for array max

I store my API key in a txt which is not in the Git. Simply define a variable api_key

In [ ]:
data = ""
with open ("ApiKey.txt", "r") as myfile:
    lines =myfile.readlines()
for l in lines:
    data = data + l.strip()
api_key = data.strip();

In [ ]:
gmaps.configure(api_key=api_key) # Your Google API key
gmapsClient = googlemaps.Client(key=api_key)

Helper function to modify a latitude / longitude position.

In [ ]:
# see https://stackoverflow.com/questions/7477003/calculating-new-longitude-latitude-from-old-n-meters
def AddToLatLng(currentPositionPair, deltaLat, deltaLong):
    earth = 6378.137
    pi = 3.14196
    m = (1 / ((2 * pi / 360) * earth)) / 1000  #1 meter in degree

    new_latitude = currentPositionPair[0] + (deltaLat * m);
    latitude =currentPositionPair[1]
    new_longitude = latitude + (deltaLong * m) / math.cos(latitude * (pi / 180));
    return (new_latitude,new_longitude)

Either define the start position by its latitude / longitude position.

In [ ]:
# use coodrinates 
startKoordinates = (52.5250871,13.3672133,17)

Or Use Google to lookup the position

In [ ]:
# retrieve coodrinates from address string
startPoint = "Berlin Hauptbahnhof"
try:
    geocode_result = gmapsClient.geocode(startPoint)
    startKoordinates = (float(geocode_result[0]['geometry']['location']['lat']), float(geocode_result[0]['geometry']['location']['lng']))
except Exception as ex:
    print("Could not resolve start position: " + str(ex))

In [ ]:
startTime = datetime.strptime('Aug 8 2018  1:00PM', '%b %d %Y %I:%M%p')

Define rectangle to scan, and random shuffle the position. This way a general overview should be generated fast, which gets more and more precices.

In [ ]:
areaSideLenInMeter = 30000
stepSizeInMeters= 50

granulatiriy= int(areaSideLenInMeter / stepSizeInMeters)
toCheck = []
locations = []
weights = []
for lng in range(-granulatiriy,granulatiriy+1):
    for lat in range(-granulatiriy,granulatiriy+1):
        positionToCheck = AddToLatLng(startKoordinates,lat*stepSizeInMeters,lng*stepSizeInMeters)
        toCheck.append(positionToCheck)
        
random.shuffle(toCheck)

 **Load Results** 

In [ ]:
infile = open("saveForLater",'rb')
loadData = pickle.load(infile)
infile.close()

toCheck= loadData["toCheck"] 
locations = loadData["locations"] 
weights = loadData["weights"] 
startTime = loadData["startTime"] 
if "startPoint" in loadData:
    startPoint = loadData["startPoint"] 
else:
    startPoint = None   
startKoordinates = loadData["startKoordinates"] 

loadData = None
print("Loctions to check: " +str(len(toCheck)))
print("Loctions already checked: " +str(len(locations)))
print("Starttime " + str(startTime) + " from " + str(startKoordinates))

Check the next **checkNext** positions for their travel time

In [ ]:
checkNext = 10000
sleepTime=2 # not sure how much my api key allows

for i in range(checkNext):
    if (len(toCheck) == 0):
        print("Finished")
        break
        
    positionToCheck = toCheck.pop()
    print(str(i) + " Check:" + str(positionToCheck))
    
    try:
        directions_result = gmapsClient.directions(startKoordinates,positionToCheck,mode="transit",departure_time=startTime)
        #print(str(directions_result))
        timeInSec = directions_result[0]['legs'][0]['duration']['value']
            
        locations.append(positionToCheck)
        weights.append(timeInSec)            
        print("Takes " + str(timeInSec/60) + " minutes")
    except Exception as ex:            
        print("Could not resolve route to position: " + str(ex))        
    time.sleep(sleepTime)                           

In [ ]:
infoString = "Travel Time Heatmap"
if (startPoint):
    infoString = infoString + " from " + startPoint + str(startKoordinates) 
else:
    infoString = infoString + " from " + str(startKoordinates)
infoString = infoString + " on " + str(startTime) + " maximum Time is " + str(numpy.amax(weights)/60) + " minutes"

print(infoString)
fig = gmaps.figure()
heatmap = gmaps.heatmap_layer(locations,weights)
#heatmap.dissipating = False
heatmap.point_radius =30
fig.add_layer(heatmap)
fig

**Save Results** for later refinement

In [ ]:
SaveData = {}
SaveData["toCheck"] = toCheck
SaveData["locations"] = locations
SaveData["weights"] = weights
SaveData["startTime"] = startTime    
SaveData["startKoordinates"] = startKoordinates

try:
    SaveData["startPoint"] = startPoint
except NameError:
    startPoint = None    

outfile = open("saveForLater",'wb')
pickle.dump(SaveData,outfile)
outfile.close()